In [1]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import losses, backend
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import tensorflow_addons as tfa
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


In [3]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

ss = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [4]:
train_features.info()
train_features.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23814 entries, 0 to 23813
Columns: 876 entries, sig_id to c-99
dtypes: float64(872), int64(1), object(3)
memory usage: 159.2+ MB


,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
1,id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
2,id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
3,id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
4,id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125


In [5]:
test_features.info()
test_features.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3982 entries, 0 to 3981
Columns: 876 entries, sig_id to c-99
dtypes: float64(872), int64(1), object(3)
memory usage: 26.6+ MB


,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380


In [6]:
train_targets.info()
train_targets.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23814 entries, 0 to 23813
Columns: 207 entries, sig_id to wnt_inhibitor
dtypes: int64(206), object(1)
memory usage: 37.6+ MB


,sig_id,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del df['sig_id']
    #df = df.drop(not_imp_features, axis=1)
    return df

train = preprocess(train_features)
test = preprocess(test_features)

#train = train.loc[train_features['cp_type']=='trt_cp'].reset_index(drop=True)
#train_targets = train_targets.loc[train_features['cp_type']=='trt_cp'].reset_index(drop=True)
del train_targets['sig_id']

print(train.shape)
print(train_targets.shape)

(23814, 875)
(23814, 206)


In [8]:
train_columns = train.columns.to_list()
g_list = [i for i in train_columns if i.startswith('g-')]
c_list = [i for i in train_columns if i.startswith('c-')]

train['mean_g-'] = train[g_list].mean(axis=1)
train['mean_c-'] = train[c_list].mean(axis=1)

test['mean_g-'] = test[g_list].mean(axis=1)
test['mean_c-'] = test[c_list].mean(axis=1)

The importance features were received via the ExtraTreesClassifier model. 

In [9]:
'''
# Feature Importance with Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier

X = train.copy()
#X_val = features_valid.copy()
Y = train_targets.copy()
#Y_val = target_valid.copy()

model = ExtraTreesClassifier(n_jobs=-1, random_state=42, verbose=2, n_estimators=300, max_depth=11)

model.fit(X, Y)
'''

'\n# Feature Importance with Extra Trees Classifier\nfrom sklearn.ensemble import ExtraTreesClassifier\n\nX = train.copy()\n#X_val = features_valid.copy()\nY = train_targets.copy()\n#Y_val = target_valid.copy()\n\nmodel = ExtraTreesClassifier(n_jobs=-1, random_state=42, verbose=2, n_estimators=300, max_depth=11)\n\nmodel.fit(X, Y)\n'

In [10]:
'''features=X.columns
importances = model.feature_importances_
indices = np.argsort(importances)
importances[indices]'''

'features=X.columns\nimportances = model.feature_importances_\nindices = np.argsort(importances)\nimportances[indices]'

In [11]:
'''# Notimportant features
notimportant_features_trees = features[indices[importances[indices]<0.00022]] 
print(list(notimportant_features_trees))
print(len(notimportant_features_trees))'''

'# Notimportant features\nnotimportant_features_trees = features[indices[importances[indices]<0.00022]] \nprint(list(notimportant_features_trees))\nprint(len(notimportant_features_trees))'

Here is a list of notimportant features that we can drop.

In [ ]:
not_imp_features = ['g-247', 'g-104', 'g-234', 'g-707', 'g-571', 'g-54', 'g-375', 'g-413', 'g-227', 'g-88', 
                    'g-341', 'g-519', 'g-45', 'g-573', 'g-549', 'g-172', 'g-438', 'g-555', 'g-232', 'g-536', 
                    'g-756', 'g-430', 'g-592', 'g-721', 'g-452', 'g-605', 'g-302', 'g-56', 'g-32', 'g-582', 
                    'g-219', 'g-109', 'g-713', 'g-358', 'g-717', 'g-376', 'g-607', 'g-101', 'g-370', 'mean_g-', 
                    'g-114', 'g-191', 'g-755', 'g-196', 'g-326', 'g-484', 'g-754', 'g-44', 'g-747', 'g-76', 
                    'g-401', 'g-223', 'g-286', 'g-86', 'g-653', 'g-111', 'g-654', 'g-513', 'g-531', 'g-2', 
                    'g-550', 'g-611', 'g-725', 'g-340', 'g-276', 'g-346', 'g-710', 'g-650', 'g-530', 'g-748', 
                    'g-532', 'g-581', 'g-24', 'g-719', 'g-331', 'g-682', 'g-739', 'g-244', 'g-82', 'g-5', 
                    'g-687', 'g-737', 'g-161', 'g-399', 'g-425', 'g-545', 'g-103', 'g-645', 'g-473', 'g-740', 
                    'g-15', 'g-681', 'g-362', 'g-483', 'g-676', 'g-612', 'g-238', 'g-585', 'g-184', 'g-680', 
                    'g-580']

In [13]:
train = train.drop(not_imp_features, axis=1)
test = test.drop(not_imp_features, axis=1)

In [14]:
print(train.shape)
print(test.shape)

(23814, 776)
(3982, 776)


In [15]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#scaler = StandardScaler()
scaler = MinMaxScaler()
num_columns = train.columns
train[num_columns] = scaler.fit_transform(train[num_columns])
test[num_columns] = scaler.transform(test[num_columns])

In [16]:
def create_model(num_columns):
    model = tf.keras.Sequential([
    tf.keras.layers.Input(num_columns),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(2048, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(1024, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.25), 
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(512, activation="relu")),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid"))
    ])
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy', 
                  metrics=logloss
                  )
    return model

In [17]:
len(train.columns)

776

In [18]:
def metric(y_true, y_pred):
    metrics = []
    for _target in train_targets.columns:
        metrics.append(log_loss(y_true.loc[:, _target], y_pred.loc[:, _target].astype(float), labels=[0,1]))
    return np.mean(metrics)


# Prediction Clipping Thresholds

p_min = 0.001
p_max = 0.999

# Evaluation Metric with clipping and no label smoothing

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -backend.mean(y_true*backend.log(y_pred) + (1-y_true)*backend.log(1-y_pred))

In [19]:
SEEDS = [23, 62, 107, 228, 404, 2208]

res = train_targets.copy()
ss.loc[:, train_targets.columns] = 0
res.loc[:, train_targets.columns] = 0
oof = tf.constant(0.0)

for seed in SEEDS:
    for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits=7, random_state=seed, shuffle=True).split(train_targets, train_targets)):
        print(f'Seed {seed}, Fold {n}')
    
        model = create_model(len(train.columns))
        checkpoint_path = f'repeat:{seed}_Fold:{n}.hdf5'
        reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, epsilon=1e-4, mode='min')
        cb_checkpt = ModelCheckpoint(checkpoint_path, monitor = 'val_loss', verbose = 0, save_best_only = True,
                                     save_weights_only = True, mode = 'min')
        model.fit(train.values[tr],
                  train_targets.values[tr],
                  validation_data=(train.values[te], train_targets.values[te]),
                  epochs=35, batch_size=192,
                  callbacks=[reduce_lr_loss, cb_checkpt], verbose=2
                 )
        
        model.load_weights(checkpoint_path)
        
        val_predict = model.predict(train.values[te]) 
        oof += logloss(tf.constant(train_targets.values[te],dtype=tf.float32),tf.constant(val_predict,dtype=tf.float32))
        res.loc[te, train_targets.columns] += val_predict
        
        test_predict = model.predict(test.values)
        ss.loc[:, train_targets.columns] += test_predict
        print('')
    
ss.loc[:, train_targets.columns] /= ((n+1) * len(SEEDS))
res.loc[:, train_targets.columns] /=  len(SEEDS)
oof /= ((n+1) * len(SEEDS))

Seed 23, Fold 0
Epoch 1/35
107/107 - 1s - loss: 0.5741 - logloss: 0.5715 - val_loss: 0.2624 - val_logloss: 0.2624
Epoch 2/35
107/107 - 1s - loss: 0.1103 - logloss: 0.1098 - val_loss: 0.0451 - val_logloss: 0.0451
Epoch 3/35
107/107 - 1s - loss: 0.0363 - logloss: 0.0362 - val_loss: 0.0283 - val_logloss: 0.0283
Epoch 4/35
107/107 - 1s - loss: 0.0260 - logloss: 0.0259 - val_loss: 0.0227 - val_logloss: 0.0227
Epoch 5/35
107/107 - 1s - loss: 0.0227 - logloss: 0.0226 - val_loss: 0.0214 - val_logloss: 0.0213
Epoch 6/35
107/107 - 1s - loss: 0.0206 - logloss: 0.0205 - val_loss: 0.0196 - val_logloss: 0.0194
Epoch 7/35
107/107 - 1s - loss: 0.0194 - logloss: 0.0194 - val_loss: 0.0187 - val_logloss: 0.0186
Epoch 8/35
107/107 - 1s - loss: 0.0185 - logloss: 0.0185 - val_loss: 0.0178 - val_logloss: 0.0178
Epoch 9/35
107/107 - 1s - loss: 0.0180 - logloss: 0.0181 - val_loss: 0.0174 - val_logloss: 0.0174
Epoch 10/35
107/107 - 1s - loss: 0.0176 - logloss: 0.0176 - val_loss: 0.0171 - val_logloss: 0.0171
Epo

In [20]:
print(f'OOF Metric: {metric(train_targets, res)}')

OOF Metric: 0.014886022634389168


In [21]:
print(oof)

tf.Tensor(0.015393173, shape=(), dtype=float32)


In [22]:
ss.loc[test_features['cp_type']=='ctl_vehicle', train_targets.columns] = 0

In [23]:
ss.to_csv('submission.csv', index=False)